In [1]:
import requests
import pandas as pd

# URL base de la API Ergast
base_url = "http://ergast.com/api/f1/2024/{}/qualifying.json"

# Lista para almacenar los resultados de clasificación
qualy_results = []

# Iterar sobre las primeras 19 carreras de la temporada (ajusta el rango si es necesario)
for round_number in range(1, 20):  # Se usa 1 a 19, ya que las rondas comienzan desde 1
    response = requests.get(base_url.format(round_number))
    
    # Asegurarse de que la respuesta sea válida
    if response.status_code == 200:
        data = response.json()
        
        # Verificar que haya datos de clasificación
        if data['MRData']['RaceTable']['Races']:
            race_data = data['MRData']['RaceTable']['Races'][0]
            race_name = race_data['raceName']
            qualifying_results = race_data['QualifyingResults']
            
            # Extraer los tiempos de vuelta y piloto
            for result in qualifying_results:
                qualy_results.append({
                    'round': round_number,
                    'race': race_name,
                    'driver': result['Driver']['givenName'] + " " + result['Driver']['familyName'],
                    'Q1_time': result.get('Q1', None),
                    'Q2_time': result.get('Q2', None),
                    'Q3_time': result.get('Q3', None)
                })
        else:
            # Si no hay resultados de clasificación para este round, agregar una entrada indicando que no se ha corrido
            qualy_results.append({
                'round': round_number,
                'race': f"Round {round_number} not yet run",
                'driver': None,
                'Q1_time': None,
                'Q2_time': None,
                'Q3_time': None
            })

# Crear un DataFrame con los resultados
df_qualy = pd.DataFrame(qualy_results)

# Mostrar el DataFrame resultante
df_qualy


,round,race,driver,Q1_time,Q2_time,Q3_time
0,1,Bahrain Grand Prix,Max Verstappen,1:30.031,1:29.374,1:29.179
1,1,Bahrain Grand Prix,Charles Leclerc,1:30.243,1:29.165,1:29.407
2,1,Bahrain Grand Prix,George Russell,1:30.350,1:29.922,1:29.485
3,1,Bahrain Grand Prix,Carlos Sainz,1:29.909,1:29.573,1:29.507
4,1,Bahrain Grand Prix,Sergio Pérez,1:30.221,1:29.932,1:29.537
...,...,...,...,...,...,...
355,18,Singapore Grand Prix,Lance Stroll,1:31.094,None,None
356,18,Singapore Grand Prix,Pierre Gasly,1:31.312,None,None
357,18,Singapore Grand Prix,Valtteri Bottas,1:31.572,None,None
358,18,Singapore Grand Prix,Guanyu Zhou,1:32.054,None,None


In [2]:
import requests
import pandas as pd

# URL base de la API Ergast
url = 'http://ergast.com/api/f1/{year}/{round}/results.json'

# Lista para almacenar los resultados de todas las carreras
gp_results = []

# Iterar sobre las primeras 3 carreras de la temporada (ajusta el rango según sea necesario)
for round_number in range(1, 25):  # Ejemplo con 3 rondas, puedes ajustarlo
    response = requests.get(url.format(year=2024, round=round_number))
    
    # Asegurarse de que la respuesta sea válida
    if response.status_code == 200:
        data = response.json()
        
        # Verificar que haya datos de carreras
        if data['MRData']['RaceTable']['Races']:
            race_data = data['MRData']['RaceTable']['Races'][0]
            race_name = race_data['raceName']
            results = race_data['Results']
            
            # Extraer los datos que te interesan para cada piloto
            for result in results:
                gp_results.append({
                    'race': race_name,
                    'number': result['number'],
                    'position': result['position'],
                    'positionText': result['positionText'],
                    'points': result['points'],
                    'driverId': result['Driver']['driverId'],
                    'driverName': f"{result['Driver']['givenName']} {result['Driver']['familyName']}",
                    'driverDOB': result['Driver']['dateOfBirth'],
                    'driverNationality': result['Driver']['nationality'],
                    'constructorId': result['Constructor']['constructorId'],
                    'constructorName': result['Constructor']['name'],
                    'constructorNationality': result['Constructor']['nationality'],
                    'grid': result['grid'],
                    'laps': result['laps'],
                    'status': result['status'],
                    'raceTime': result.get('Time', {}).get('time', None),  # Valor por defecto si no existe
                    'fastestLapTime': result.get('FastestLap', {}).get('Time', {}).get('time', None),
                    'fastestLapSpeed': result.get('FastestLap', {}).get('AverageSpeed', {}).get('speed', None)
                })

# Convertir la lista de diccionarios en un DataFrame de Pandas
df_results = pd.DataFrame(gp_results)

# Mostrar el DataFrame resultante
df_results

,race,number,position,positionText,points,driverId,driverName,driverDOB,driverNationality,constructorId,constructorName,constructorNationality,grid,laps,status,raceTime,fastestLapTime,fastestLapSpeed
0,Bahrain Grand Prix,1,1,1,26,max_verstappen,Max Verstappen,1997-09-30,Dutch,red_bull,Red Bull,Austrian,1,57,Finished,1:31:44.742,1:32.608,210.383
1,Bahrain Grand Prix,11,2,2,18,perez,Sergio Pérez,1990-01-26,Mexican,red_bull,Red Bull,Austrian,5,57,Finished,+22.457,1:34.364,206.468
2,Bahrain Grand Prix,55,3,3,15,sainz,Carlos Sainz,1994-09-01,Spanish,ferrari,Ferrari,Italian,4,57,Finished,+25.110,1:34.507,206.156
3,Bahrain Grand Prix,16,4,4,12,leclerc,Charles Leclerc,1997-10-16,Monegasque,ferrari,Ferrari,Italian,2,57,Finished,+39.669,1:34.090,207.069
4,Bahrain Grand Prix,63,5,5,10,russell,George Russell,1998-02-15,British,mercedes,Mercedes,German,3,57,Finished,+46.788,1:35.065,204.946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,Singapore Grand Prix,77,16,16,0,bottas,Valtteri Bottas,1989-08-28,Finnish,sauber,Sauber,Swiss,19,61,+1 Lap,None,1:37.524,182.355
355,Singapore Grand Prix,10,17,17,0,gasly,Pierre Gasly,1996-02-07,French,alpine,Alpine F1 Team,French,18,61,+1 Lap,None,1:36.927,183.478
356,Singapore Grand Prix,3,18,18,0,ricciardo,Daniel Ricciardo,1989-07-01,Australian,rb,RB F1 Team,Italian,16,61,+1 Lap,None,1:34.486,188.218
357,Singapore Grand Prix,20,19,19,0,kevin_magnussen,Kevin Magnussen,1992-10-05,Danish,haas,Haas F1 Team,American,14,57,Accident,None,1:37.425,182.540


In [3]:
import requests
import pandas as pd

# URLs de la API para la clasificación de pilotos y de constructores
drivers_standings_url = 'http://ergast.com/api/f1/{year}/driverStandings.json'

# Año para el que deseas obtener la clasificación
year = 2024

# Lista para almacenar la clasificación de pilotos y constructores
drivers_standings = []
constructors_standings = []

# -------------------- Clasificación de pilotos -------------------- #
response_drivers = requests.get(drivers_standings_url.format(year=year))

# Asegurarse de que la respuesta sea válida
if response_drivers.status_code == 200:
    data_drivers = response_drivers.json()
    
    # Verificar que haya datos de clasificación
    if data_drivers['MRData']['StandingsTable']['StandingsLists']:
        standings_list = data_drivers['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']
        
        # Extraer datos de los pilotos
        for standing in standings_list:
            drivers_standings.append({
                'position': standing['position'],
                'points': standing['points'],
                'wins': standing['wins'],
                'driverId': standing['Driver']['driverId'],
                'driverName': f"{standing['Driver']['givenName']} {standing['Driver']['familyName']}",
                'driverDOB': standing['Driver']['dateOfBirth'],
                'driverNationality': standing['Driver']['nationality'],
                'constructorName': standing['Constructors'][0]['name']
            })

df_drivers_standings = pd.DataFrame(drivers_standings)
df_drivers_standings

,position,points,wins,driverId,driverName,driverDOB,driverNationality,constructorName
0,1,331,7,max_verstappen,Max Verstappen,1997-09-30,Dutch,Red Bull
1,2,279,3,norris,Lando Norris,1999-11-13,British,McLaren
2,3,245,2,leclerc,Charles Leclerc,1997-10-16,Monegasque,Ferrari
3,4,237,2,piastri,Oscar Piastri,2001-04-06,Australian,McLaren
4,5,190,1,sainz,Carlos Sainz,1994-09-01,Spanish,Ferrari
5,6,174,2,hamilton,Lewis Hamilton,1985-01-07,British,Mercedes
6,7,155,1,russell,George Russell,1998-02-15,British,Mercedes
7,8,144,0,perez,Sergio Pérez,1990-01-26,Mexican,Red Bull
8,9,62,0,alonso,Fernando Alonso,1981-07-29,Spanish,Aston Martin
9,10,24,0,hulkenberg,Nico Hülkenberg,1987-08-19,German,Haas F1 Team


In [4]:
constructors_standings_url = 'http://ergast.com/api/f1/{year}/constructorStandings.json'
response_constructors = requests.get(constructors_standings_url.format(year=year))
year = 2024

# Asegurarse de que la respuesta sea válida
if response_constructors.status_code == 200:
    data_constructors = response_constructors.json()
    
    # Verificar que haya datos de clasificación
    if data_constructors['MRData']['StandingsTable']['StandingsLists']:
        standings_list = data_constructors['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']
        
        # Extraer datos de los constructores
        for standing in standings_list:
            constructors_standings.append({
                'position': standing['position'],
                'points': standing['points'],
                'wins': standing['wins'],
                'constructorId': standing['Constructor']['constructorId'],
                'constructorName': standing['Constructor']['name'],
                'constructorNationality': standing['Constructor']['nationality']
            })

# Crear DataFrames con la información extraída
df_constructors_standings = pd.DataFrame(constructors_standings)

df_constructors_standings

,position,points,wins,constructorId,constructorName,constructorNationality
0,1,516,5,mclaren,McLaren,British
1,2,475,7,red_bull,Red Bull,Austrian
2,3,441,3,ferrari,Ferrari,Italian
3,4,329,3,mercedes,Mercedes,German
4,5,86,0,aston_martin,Aston Martin,British
5,6,34,0,rb,RB F1 Team,Italian
6,7,31,0,haas,Haas F1 Team,American
7,8,16,0,williams,Williams,British
8,9,13,0,alpine,Alpine F1 Team,French
9,10,0,0,sauber,Sauber,Swiss


In [5]:
circuitdata = {
    'Round': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
    'Date': ['02 de marzo', '09 de marzo', '24 de marzo', '7 de abril', '21 de abril', '05 de mayo', '19 de mayo', 
             '26 de mayo', '09 de junio', '23 de junio', '30 de junio', '07 de julio', '21 julio', '28 julio', 
             '25 de agosto', '01 septiembre', '15 septiembre', '22 de septiembre', '20 de octubre', '27 octubre', 
             '3 de noviembre', '24 de noviembre', '01 diciembre'],
    'Country': ['Bahrain', 'Saudi Arabia', 'Australia', 'Japan', 'China', 'United States', 'Italy', 'Monaco', 
                'Canada', 'Spain', 'Austria', 'United Kingdom', 'Hungary', 'Belgium', 'Netherlands', 'Italy', 
                'Azerbaijan', 'Singapore', 'United States', 'Mexico', 'Brazil', 'United States', 'Qatar'],
    'City': ['Sakhir', 'Yeda', 'Melbourne', 'Suzuka', 'Shanghai', 'Miami', 'Imola', 'Mónaco', 'Montreal', 
             'Barcelona', 'Red Bull Ring', 'Silverstone', 'Budapest', 'Spa-Francorchamps', 'Zandvoort', 'Monza', 
             'Bakú', 'Singapur', 'Austin', 'Ciudad de México', 'Interlagos', 'Las Vegas', 'Losail']
}

# Crear DataFrame
df_circuit = pd.DataFrame(circuitdata)
df_circuit = pd.merge(df_circuit, df_results, on='round', how='left')
df_circuit


KeyError: 'round'

In [ ]:
# Eliminar duplicados de la columna 'race'
df_race = df_results['race'].drop_duplicates().reset_index(drop=True)

# Crear la columna 'Round' numerando las carreras (GPs) desde 1
df_race = pd.DataFrame(df_race)  # Convertir a DataFrame si aún no lo es
df_race['Round'] = range(1, len(df_race) + 1)


# Ordenar los datos por la columna 'Round' si es necesario
df_race = df_race.sort_values('Round').reset_index(drop=True)

# Mostrar el DataFrame resultante
df_race


In [ ]:
df_combined